In [64]:
# Imports
import torch
import torchvision # torch package for vision related things
import torch.nn.functional as F  # Parameterless functions, like (some) activation functions
import torchvision.datasets as datasets  # Standard datasets
import torchvision.transforms as transforms  # Transformations we can perform on our dataset for augmentation
from torch import optim  # For optimizers like SGD, Adam, etc.
from torch import nn  # All neural network modules
from torch.utils.data import DataLoader  # Gives easier dataset managment by creating mini batches etc.
from tqdm import tqdm  # For nice progress bar!


In [55]:
device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [65]:


class NN(nn.Module):
    def __init__(self, input_size, num_classes):
        super(NN, self).__init__()
        # self.fc1=nn.Linear(input_size, 200)
        # self.fc2=nn.Linear(200, 100)
        # self.fc3=nn.Linear(100, 50)
        # self.fc4=nn.Linear(50, num_classes)
        self.fc1 = nn.Linear(input_size, 50)
        self.fc2 = nn.Linear(50, num_classes)
    
    def forward(self, x):
        # x=F.relu(self.fc1(x))
        # x=F.relu(self.fc2(x))
        # x=F.relu(self.fc3(x))
        # x=self.fc4(x)
        x=F.relu(self.fc1(x))
        x=self.fc2(x)
        return x


In [66]:
# Hyperparamters

input_size=784
num_classes=10
learning_rate=0.001
batch_size=64
num_epochs=5


In [67]:
# Loading dataset
train_dataset=datasets.MNIST(root='datasets/',train=True,transform=transforms.ToTensor(),download=True)
train_loader=DataLoader(dataset=train_dataset,batch_size=batch_size,shuffle=True)

test_dataset=datasets.MNIST(root='datasets/',train=False,transform=transforms.ToTensor(),download=True)
test_loader=DataLoader(dataset=test_dataset,batch_size=batch_size,shuffle=True)



In [71]:
model=NN(input_size=input_size,num_classes=num_classes).to(device=device)

In [72]:
#Loss and Optimizer

criterion=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(),lr=learning_rate)


In [78]:
def check_accuracy(loader,model):
    num_correct=0
    num_samples=0
    model.eval()  # to turn off batch normalisation and dropout layers

    with torch.no_grad():  # to turn off backprobagation
        for x, y in loader:
            x=x.to(device=device)
            x=x.reshape((x.shape[0],-1))

            y=torch.tensor(y).to(device=device)

            scores=model(x)
            _, predictions=scores.max(1)  #max return a tuple (max element, max element index)
            num_correct+= (predictions==y).sum()
            num_samples+= predictions.size(0)
        #print('accuracy=',((float(num_correct)/num_samples)*100) )

    model.train()   # to return model back to training mode
    return (float(num_correct)/num_samples)*100

In [79]:
# Train Network



for epoch in range(num_epochs):
    for batch_idx, (data,targets) in enumerate(tqdm(train_loader)):

        #getting the data
        data=data.to(device=device)
        targets=targets.to(device=device)
        # data is the batch of data, targets is the target label of each sample in the batch

        # reshaping the data to change from a 2d image to a 1d vector
        data=data.reshape((data.shape[0],-1))
        #data=data.to(device=device)

        #forward
        scores=model(data)
        loss=criterion(scores,targets)
        
        torch.set_grad_enabled(True)  # Context-manager 

        #backward
        optimizer.zero_grad()
        loss.backward()

        #gradient descent
        optimizer.step()
    print('epoch_num=',epoch,'   Train accuracy=',check_accuracy(model=model,loader=train_dataset))
    print('=======================================================')

        

#check_accuracy(model,test_dataset)







100%|██████████| 938/938 [00:09<00:00, 103.51it/s]


epoch_num= 0    Train accuracy= 96.83666666666667


100%|██████████| 938/938 [00:09<00:00, 102.94it/s]


epoch_num= 1    Train accuracy= 97.03


100%|██████████| 938/938 [00:09<00:00, 101.40it/s]


epoch_num= 2    Train accuracy= 97.76166666666667


100%|██████████| 938/938 [00:09<00:00, 102.74it/s]


epoch_num= 3    Train accuracy= 97.85000000000001


100%|██████████| 938/938 [00:09<00:00, 101.05it/s]


epoch_num= 4    Train accuracy= 98.22666666666666


In [81]:
print('test accuracy=',check_accuracy(loader=test_dataset,model=model))

test accuracy= 97.05


tensor([[1, 1],
        [2, 2]], device='cuda:0')
